In [1]:
from keras.preprocessing.text import Tokenizer
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

sns.set()

Using TensorFlow backend.


In [2]:
categories = pd.read_csv('./categories.csv')
categories_map = pd.read_excel('./category_mapping.xlsx', 'Sheet1')
new_article = pd.read_excel('./news_details.xlsx', 'Sheet1')

In [3]:
new_article['content'] = new_article['title'].str.cat(new_article['snippet'], na_rep = '', sep = "\n").str.cat(new_article['news_description'], na_rep = '', sep = '\n')

In [4]:
new_article = new_article.merge(categories_map, on = "news_id")

In [5]:
data = new_article[['content', 'category_id']]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data['content'], data['category_id'], test_size = 0.2)

In [7]:
def clean_text(text):
    text = text.replace('\\n', '')
    text = re.sub('[^A-Za-z-]', ' ', text)
    text = [word for word in text.split() if len(word) > 2]
    return ' '.join(text)

#### Convert to text to tokenizer

In [8]:
num_classes = categories.shape[0]
max_words = 100000
tokenizer = Tokenizer(num_words= max_words)
tokenizer.fit_on_texts(x_train)

In [9]:
train_tfidf = tokenizer.texts_to_matrix(x_train)
test_tfidf = tokenizer.texts_to_matrix(x_test)

In [10]:
## Converting class label into One hot encoded vector

y_train = to_categorical(y_train - 1, num_classes)
y_test = to_categorical(y_test - 1, num_classes)

In [11]:
## Creating Simple Neural network

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [13]:
model = Sequential()
model.add(Dense(1024, activation = 'relu', input_shape = (max_words,)))
model.add(Dropout(0.2))
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              102401024 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 7175      
_________________________________________________________________
activation_1 (Activation)    (None, 7)                 0         
Total params: 103,457,799
Trainable params: 103,457,799
Non-trainable params: 0
________________________________________

In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
history = model.fit(train_tfidf, y_train, batch_size = 128, epochs = 5, verbose = 1, validation_split = 0.1)

Train on 3877 samples, validate on 431 samples
Epoch 1/5
3877/3877 [==============================] - 91s 23ms/step - loss: 0.7061 - accuracy: 0.7728 - val_loss: 0.2753 - val_accuracy: 0.9234
Epoch 2/5
3877/3877 [==============================] - 87s 22ms/step - loss: 0.0413 - accuracy: 0.9925 - val_loss: 0.3417 - val_accuracy: 0.9142
Epoch 3/5
3877/3877 [==============================] - 61s 16ms/step - loss: 0.0181 - accuracy: 0.9982 - val_loss: 0.3952 - val_accuracy: 0.9026
Epoch 4/5
3877/3877 [==============================] - 60s 16ms/step - loss: 0.0117 - accuracy: 0.9987 - val_loss: 0.4511 - val_accuracy: 0.8910
Epoch 5/5
3877/3877 [==============================] - 62s 16ms/step - loss: 0.0083 - accuracy: 0.9987 - val_loss: 0.4422 - val_accuracy: 0.9049
